In [ ]:
import pandas as pd
import requests
import notebook_utils.notebook_helpers as utils

# Priority Routing
## Capacitated Vehicle Routing Problem with Soft Time Windows (CVRPSTW)

Loyalty (or Preferred) customer programs help companies to reward repeat customers and enhance their overall business offering. While the best possible customer service is always the goal, loyalty programs provide a mechanism for reinforcing the relationship with the customers that drive business revenue.

In this scenario we have a set of deliveries with target time windows for delivery that do not represent a feasible solution given the delivery vehicles that are available. We would still like to deliver all the packages even if some of them are a little behind schedule. However, we would like to prioritize the deliveries of customers in our loyalty program to minimize the delay these customers experience.

We also want to optimize according to a business defined cost objective that is a combination of business relevant metrics. To track time window constraints we will pass a time matrix as a constraint checking "secondary matrix".


### Problem Details:
- 8 Locations each with an associated demand
    - 1 Distribution Center 
        - distribution center demand: [0]
        - hours of operation: [0,24]
    - 7 Service Locations
        - demand for deliveries: [1, 1, 1, 1, 1, 1, 1]
        - delivery time windows: [[9,10],[9,10],[9,10],[10,11],[10,11],[10,11],[9,10]]
        - service location service times: [ 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
        - loyalty program member: [1, 0, 0, 0, 1, 0, 1]

- 3 Delivery vehicles each with an associated capacity
    - 3 delivery vehicles
        - capacity for deliveries: [3, 3, 3]
        
**API Reference**: [cuOpt Server Documentation](https://docs.nvidia.com/cuopt/serv_api.html)

Below we visualize the delivery locations with respect to the distribution center. The cost from all locations to all other locations (a cost matrix) will be required for optimization. To see an example of cost matrix generation from map data or a waypoint graph, refer to the [cost_matrix_creation.ipynb](cost_matrix_creation.ipynb) notebook.  For the purpose of this simple example we will omit the cost matrix calculation.

In [ ]:
location_names       = [      "DC",          "A",    "B",    "C",    "D",    "E",    "F",    "G"  ]
location_coordinates = [     [4, 4],        [1, 3], [8, 1], [2, 1], [6, 7], [0, 2], [7, 6], [5, 3] ]
location_ids         = [      i for i in range(len(location_names))]
location_coordinates_df = pd.DataFrame(location_coordinates, columns=['xcord', 'ycord'], index=location_names)
utils.gen_plot(location_coordinates_df).show()

## Setup the cuOpt server and test its health

**NOTE**: Please update **ip** and **port** on which the server is running.

In [ ]:
ip = "0.0.0.0"
port = "5000"
url = "http://" + ip + ":" + port + "/cuopt/"

# Test server health
assert requests.get(url + "health").status_code == 200

### Cost Matrix : Primary

The cost matrix dictates the cost of travel between locations of interest.  The cost itself can be anything relevant to the user.  In this case we are using a business defined cost objective as a primary cost matrix and a secondary time matrix to verify our time based constraints. 

Here is the cost(business metric) matrix corresponding to the locations above:

In [ ]:
business_metric_cost_matrix = [
    [0.0, 3.1, 5.0, 3.6, 3.6, 4.5, 3.6, 1.4],
    [3.1, 0.0, 7.3, 2.2, 6.4, 1.4, 6.7, 4.0],
    [5.0, 7.3, 0.0, 6.0, 6.3, 8.1, 5.1, 3.6],
    [3.6, 2.2, 6.0, 0.0, 7.2, 2.2, 7.1, 3.6],
    [3.6, 6.4, 6.3, 7.2, 0.0, 7.8, 1.4, 4.1],
    [4.5, 1.4, 8.1, 2.2, 7.8, 0.0, 8.1, 5.1],
    [3.6, 6.7, 5.1, 7.1, 1.4, 8.1, 0.0, 3.6],
    [1.4, 4.0, 3.6, 3.6, 4.1, 5.1, 3.6, 0.0]
]

### Cost Matrix : Secondary

Here is the constraint checking (time) secondary matrix:

In [ ]:
constraint_checking_time_matrix = [
    [0.00, 0.39, 0.63, 0.45, 0.45, 0.55, 0.45, 0.18 ],
    [0.39, 0.00, 0.90, 0.28, 0.80, 0.18, 0.84, 0.50 ],
    [0.63, 0.90, 0.00, 0.75, 0.79, 1.00, 0.64, 0.45 ],
    [0.45, 0.28, 0.75, 0.00, 0.90, 0.28, 0.88, 0.45 ],
    [0.45, 0.80, 0.79, 0.90, 0.00, 0.96, 0.18, 0.51 ],
    [0.55, 0.18, 1.00, 0.28, 0.96, 0.00, 1.00, 0.64 ],
    [0.45, 0.84, 0.64, 0.88, 0.18, 1.00, 0.00, 0.45 ],
    [0.18, 0.50, 0.45, 0.45, 0.51, 0.64, 0.45, 0.00 ]
]

### Deliveries

Setup the delivery data

In [ ]:
delivery_location_data = {
    "location_ids": location_names,
    "delivery_demand":        [0,  1,  1,  1,  1,  1,  1,  1 ],
    "location_earliest_time": [5,  9,  9,  9,  10, 10, 10, 9 ],
    "location_latest_time":   [20, 10, 10, 10, 11, 11, 11, 10],
    "required_service_time":  [0,  1,  1,  1,  1,  1,  1,  1 ],
    "loyalty_member":         [0,  0,  1,  0,  1,  0,  1,  0 ]
}

### Set Cost Matrix

Dispatch cost matrix to server

In [ ]:
data_params = {"return_data_state": False}
cost_data = {"cost_matrix": business_metric_cost_matrix}
response_set = requests.post(
    url + "set_cost_matrix", params=data_params, json=cost_data
)
assert response_set.status_code == 200

### Set Secondary Cost Matrix

Dispatch secondary cost matrix to server

In [ ]:
# set the secondary constraint checking time matrix
time_data = {"cost_matrix": constraint_checking_time_matrix}
response_set = requests.post(
    url + "set_secondary_matrix", params=data_params, json=time_data
)
assert response_set.status_code == 200

### Set Vehicle Data
Dispatch vehicle data to server

In [ ]:
n_vehicles = 3
vehicle_capacity = 3 # As per problem statement, all vehicles have capacities of 3

# Build the fleet data
fleet_data = {
    # Vehicle start and end at location 0, since 0 is distribution center
    "vehicle_locations": [[0,0]] * n_vehicles,
    "capacities": [[vehicle_capacity] * n_vehicles],
}

# Dispatch the fleet data request to the server
response_set = requests.post(url + "set_fleet_data", json=fleet_data)
assert response_set.status_code == 200

### Set Task Data
Dispatch task data to server

In [ ]:
# Build the task data
task_data = {
    "task_locations": location_ids,
    "demand": [delivery_location_data["delivery_demand"]],
}

# add time window constraints and service time for the locations
task_data["task_time_windows"] = list(zip(delivery_location_data["location_earliest_time"],
                                          delivery_location_data["location_latest_time"]))

task_data["service_times"] = delivery_location_data["required_service_time"]

# Dispatch the task request to the server
response_set = requests.post(url + "set_task_data", json=task_data)
assert response_set.status_code == 200

### Set CuOpt Solver Configuration

In [ ]:
# Setup the solver settings json datastructure
solver_settings = {
    # solver_settings will run for given time limit.  Larger and/or more complex problems may require more time.
    "time_limit": 0.05,
    # set number of climbers that will try to search for an optimal routes in parallel
    "number_of_climbers": 128,
}

# dispatch the solver settings to the server
response_set = requests.post(
    url + "set_solver_config", json=solver_settings
)
assert response_set.status_code == 200

### Attempt to obtain optimized routes
We can attempt to solve this problem as stated, but as previously discussed it is not feasible within the specified target time windows

In [ ]:
solver_response = requests.get(url + "get_optimized_routes")

if solver_response.status_code == 200:
    print("Cost for the routing in time: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_response.text)

cuOpt is unable to find a feasible solution.  As previously discussed we would like to allow the deliveries to exceed the latest time windows by using soft time windows

### Initial Solution

With soft time window option, we can relax time window constraints along with penality to come up with a solution but at a additional cost. 

#### Update solver configuration to use Soft Time windows

In [ ]:
solver_settings["solution_scope"] = 'SOFT_TW'
# Update the solver settings to the server
response_set = requests.put(
    url + "update_solver_config", json=solver_settings
)
assert response_set.status_code == 200

#### Add Penalty

With this, we can prioritize order/customers by providing higher penalties to such jobs compared to others.

In [ ]:
delivery_location_data['penalty'] = [x * 100 for x in delivery_location_data["loyalty_member"]]
delivery_location_data


task_data = {
    "penalties": delivery_location_data["penalty"]
}

# Update the task request to the server
response_set = requests.put(url + "update_task_data", json=task_data)
assert response_set.status_code == 200

#### Re-optimize 

In [ ]:
solver_response = requests.get(url + "get_optimized_routes")
solver_resp = solver_response.json()["response"]["solver_response"]
if solver_resp["status"] == 0: 
    solver_resp_df = utils.create_solution_dataframe(solver_resp)
    print("Cost for the routing in time: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    print(solver_resp_df)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

In [ ]:
solution_data_priority = utils.create_solution_dataframe(solver_resp)
solution_data_priority['route'] = [location_names[i] for i in solution_data_priority['route'].to_list()]
solution_data_priority = solution_data_priority.set_index('route')
solution_data_priority

_____

#### SPDX-FileCopyrightText: Copyright (c) 2022 NVIDIA CORPORATION & AFFILIATES. All rights reserved.

#### SPDX-License-Identifier: MIT

Permission is hereby granted, free of charge, to any person obtaining a
copy of this software and associated documentation files (the "Software"),
to deal in the Software without restriction, including without limitation
the rights to use, copy, modify, merge, publish, distribute, sublicense,
and/or sell copies of the Software, and to permit persons to whom the
Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
DEALINGS IN THE SOFTWARE.

---